In [2]:
import cv2
import time
import threading
import numpy as np

cap = cv2.VideoCapture('img/Car.MOV') # video path
after_validation_img = None

if __name__=="__main__":
    while(cap.isOpened()):
        ret, frame = cap.read()
        if (frame is None):
            print("End")
            break
        _frame = cv2.resize(frame, (960, 540)) # resize the frame to fit the screen
        frame_height, frame_width = frame.shape[:2]
        _frame_height, _frame_width = _frame.shape[:2]
        
        cv2.imshow('video', _frame)
        
        imgBlurred = cv2.GaussianBlur(_frame, (7, 7), 0)
        imgBlurred = cv2.GaussianBlur(imgBlurred, (7, 7), 0)
        # convert to gray
        gray = cv2.cvtColor(imgBlurred, cv2.COLOR_BGR2GRAY)
        # sobelX to get the vertical edges
        sobelx = cv2.Sobel(gray, cv2.CV_8U, 1, 0, ksize=3) 
        # 오츠알고리즘
        ret2, threshold_img = cv2.threshold(sobelx, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        # 모폴로지 커널 사이즈
        element = cv2.getStructuringElement(shape=cv2.MORPH_RECT, ksize=(20, 3))
        morph_img_threshold = threshold_img.copy()
        cv2.morphologyEx(src=threshold_img, op=cv2.MORPH_CLOSE, kernel=element, dst=morph_img_threshold)
        cv2.imshow('morphed', morph_img_threshold)
        cv2.waitKey(20)
        
        # 컨투어 추출
        _, extracted_contours, _ = cv2.findContours(morph_img_threshold, mode=cv2.RETR_EXTERNAL, method=cv2.CHAIN_APPROX_NONE)
        def check_rect(rect):
            minArea = 4500
            maxArea = 10000
            ratioMin = 2
            ratioMax = 6
            (x, y), (width, height), rect_angle = rect
            # angle preprocess
            if (width > height):
                angle = -rect_angle
            else:
                angle = 90 + rect_angle
            # 앵글이 15도 이상 틀어져있으면 False
            if angle > 15:
                return False

            #rect 넓이 저장
            area = width*height
            #ratio 범위 1:2.5 ~ 1:7
            try:
                ratio = float(width)/float(height)
            except:
                pass
            # 범위밖 이거나, 비율이 다를경우 False 리턴
            if (area < minArea or area > maxArea) or (ratio < ratioMin or ratio > ratioMax):
                return False
            return True
        
        # minAreaRect 이용
        for cnts in extracted_contours:
            min_rect = cv2.minAreaRect(cnts)
            # 컨투어를 둘러싸는 박스 찾기
            if check_rect(min_rect):
                x, y, w, h = cv2.boundingRect(cnts)
                after_validation_img = _frame[y:y+h, x:x+w]
                cv2.imshow('plate', after_validation_img)
                _frame_copy = _frame.copy()
                cv2.drawContours(_frame_copy, cnts, -1, (0,255,0), 2)
                cv2.imshow('morph cont', _frame_copy)
            else:
                after_validation_img = None
q
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()